In [1]:
##########################
# Knee Bend Pose Problem #
#    Siddhartha Dubey    #
# thesiddubey@gmail.com  #
##########################

In [2]:
import cv2
import mediapipe as mp
import os
import numpy as np
import math 
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from IPython.display import Video
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
def findDistance(x1,y1,x2,y2):
    dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dist

In [6]:
def findAngles(x1,y1,x2,y2):
    theta = math.acos((-y1)*(y1-y2)/(math.sqrt((x2-x1)**2 + (y2-y1)**2) * y1))
    degree = int(180/math.pi) * theta
    return degree

In [8]:
# Initialize frame counters.
good_frames = 0
bad_frames  = 0

good_posture_counter = 0

# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX
 
# Colors.
blue = (255, 127, 0)
red = (49, 49, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (0, 128, 0)
yellow = (0, 191, 255)
pink = (255, 0, 255)


In [9]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(model_complexity=2, min_tracking_confidence = 0.9)

input_file = "../KneeBendVideo.mp4"
cap = cv2.VideoCapture(input_file)

# Meta data
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Video writer.
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
while cap.isOpened():
    ret, image = cap.read()

    if not ret: 
        print("No Frames")
        break
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    keypoints = pose.process(image)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    lm = keypoints.pose_landmarks
    if lm is None: continue
    lmPose = mp_pose.PoseLandmark

    l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
    l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

    l_knee_x = int(lm.landmark[lmPose.LEFT_KNEE].x * w)
    l_knee_y = int(lm.landmark[lmPose.LEFT_KNEE].y * h)

    l_ankle_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * w)
    l_ankle_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * h)
    
    #Calculate angles
    hip_incline = findAngles(l_hip_x, l_hip_y, l_knee_x, l_knee_y) - 90
    ankle_incline = 90 - findAngles(l_knee_x, l_knee_y, l_ankle_x, l_ankle_y) 
    
    #Find knee angle value using angle sum property
    knee_value = 180 - hip_incline - ankle_incline
    
    #Plot Circles
    cv2.circle(image, (l_hip_x,l_hip_y), 7, yellow, -1)
    cv2.circle(image, (l_knee_x,l_knee_y), 7, yellow, -1)
    cv2.circle(image, (l_ankle_x,l_ankle_y), 7, yellow, -1)

    hip_text = 'Hip Angle: ' + str(int(hip_incline))
    ankle_text = 'Ankle Angle: ' + str(int(ankle_incline))
    knee_text = 'Knee Angle: ' + str(int(knee_value))
    
    if knee_value < 140:
        good_frames += 1
        bad_frames = 0
        
        # Output angle
        #cv2.putText(image, hip_text, (10,50), font, 0.9, light_green, 2)
        #cv2.putText(image, ankle_text, (10,80), font, 0.9, light_green, 2)
        cv2.putText(image, knee_text, (10,110), font, 0.9, light_green, 2)
        
        # Join Landmarks
        cv2.line(image, (l_hip_x, l_hip_y), (l_knee_x,l_knee_y), green, 4)
        cv2.line(image, (l_knee_x,l_knee_y), (l_ankle_x, l_ankle_y), green, 4)
    else:
        good_frames = 0
        bad_frames += 1
        
        # Output angle
        #cv2.putText(image, hip_text, (10,50), font, 0.9, light_green, 2)
        #cv2.putText(image, ankle_text, (10,80), font, 0.9, light_green, 2)
        cv2.putText(image, knee_text, (10,110), font, 0.9, light_green, 2)
        
        # Join Landmarks
        cv2.line(image, (l_hip_x, l_hip_y), (l_knee_x,l_knee_y), green, 4)
        cv2.line(image, (l_knee_x,l_knee_y), (l_ankle_x, l_ankle_y), green, 4)
        
    
    #Calculate the time of remaining in particular posture
    good_time = (1/fps) * good_frames
    bad_time = (1/fps) * bad_frames
    
    if good_time > 0:
        good_time_string = "Good Posture time: " + str(round (good_time, 1)) + 's'
        cv2.putText(image, good_time_string, (10,h-30), font, 0.9, green, 2)
        
        if good_time == 8:
            good_posture_counter += 1         
        
    else:
        keep_bent_string = "Keep your knee bent!"
        cv2.putText(image, keep_bent_string, (10,h-30), font, 1.5, red, 4)

    posture_count = "Number of good postures: "+str(good_posture_counter)
    cv2.putText(image, posture_count, (10,140), font, 0.9, yellow, 2)
    
    video_output.write(image)
    
print("Finished")
cap.release()
video_output.release()

No Frames
Finished
